In [1]:
from dh2loop import dh2l_db
import datetime


C:\ProgramData\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\00103098\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\00103098\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\00103098\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
%run -i "DH2_LConfig.py"

Default parameters loaded from DH2_LConfig.py:
#Extents to query 
#ranee
minlong=115.5
maxlong=118
minlat=-30.5
maxlat=-27.5

#fabilea
#minlong=121.2
#maxlong=122.89
#minlat=-21.04
#maxlat=-21.03


#src_pro,Dst_proj
src_csr = 4326 
dst_csr = 28350


#ExportFiles
export_path='../data/export_db/'
DB_Collar_Export=export_path+'DB_Collar_Export.csv'
DB_Survey_Export=export_path+'DB_Survey_Export.csv'
DB_Survey_Export_Calc=export_path+'DB_Survey_Export_Calc.csv'
CET_Litho=export_path+'CET_Litho.csv'
DB_Lithology_Export=export_path+'DB_Lithology_Export.csv'
DB_Lithology_Export_Backup=export_path+'DB_Lithology_Export_Backup.csv'
DB_Lithology_Upscaled_Export=export_path+'DB_Lithology_Upscaled.csv'
Upscaled_Litho_NoDuplicates_Export = export_path+'DB_Lithology_Upscaled_Export'
DB_Lithology_Export_Calc=export_path+'DB_Lithology_Export_Calc.csv'
DB_Lithology_Export_VTK=export_path+'DB_Lithology_Export.vtp'

print('Default parameters loaded from DH2_LConfig.py:')
with open('../notebooks/DH2_LConfi

## Collar Extraction

In [3]:
Start_Time = datetime.datetime.now()
dh2l_db.collar_attr_col_dic()
dh2l_db.collar_collar_attri_Final(DB_Collar_Export,src_csr,dst_csr,minlong,maxlong,minlat,maxlat)
End_Time = datetime.datetime.now()
print("Time taken in Hour_Min_Sec_MilliSec is:", End_Time-Start_Time)

Time taken in Hour_Min_Sec_MilliSec is: 0:00:47.678136


## Survey Extraction

In [4]:
Start_Time = datetime.datetime.now()
dh2l_db.Attr_col_dic()
dh2l_db.Survey_Final(DB_Survey_Export,minlong,maxlong,minlat,maxlat)
dh2l_db.convert_survey(DB_Collar_Export,DB_Survey_Export,DB_Survey_Export_Calc)
End_Time = datetime.datetime.now()
print("Time taken in Hour_Min_Sec_MilliSec is:", End_Time-Start_Time)

Time taken in Hour_Min_Sec_MilliSec is: 0:01:11.998196


## Lithology Extraction

In [3]:
Start_Time = datetime.datetime.now()
dh2l_db.Attr_Val_Dic()
dh2l_db.Litho_Dico()
dh2l_db.Clean_Up()
dh2l_db.Attr_val_With_fuzzy()
dh2l_db.Final_Lithology(DB_Lithology_Export,minlong,maxlong,minlat,maxlat)
dh2l_db.Upscale_lithology(DB_Lithology_Export,DB_Lithology_Upscaled_Export)
dh2l_db.Remove_duplicates_Litho(DB_Lithology_Upscaled_Export,Upscaled_Litho_NoDuplicates_Export)
End_Time = datetime.datetime.now()
print("Time taken in Hour_Min_Sec_MilliSec is:", End_Time-Start_Time)

First Filter ready
Time taken in Hour_Min_Sec_MilliSec is: 0:15:35.754143


In [6]:
Start_Time = datetime.datetime.now()
#dh2l_db.convert_lithology()
#dh2l_db.Diff_XYZ()
End_Time = datetime.datetime.now()
print("Time taken in Hour_Min_Sec_MilliSec is:", End_Time-Start_Time)


Time taken in Hour_Min_Sec_MilliSec is: 0:00:00
